In [147]:
import pandas as pd
from datetime import datetime, timedelta

product = pd.read_csv("D:\合作店铺\科帕奇\Agg Data\科帕奇_产品分析.csv",index_col=0)
product['dates'] = pd.to_datetime(product['year'].astype(str) + '-' + product['month'].astype(str)).dt.strftime('%Y-%m')

<>:4: SyntaxWarning: invalid escape sequence '\A'
<>:4: SyntaxWarning: invalid escape sequence '\A'
C:\Users\jackyzhu\AppData\Local\Temp\ipykernel_45836\687023429.py:4: SyntaxWarning: invalid escape sequence '\A'
  product = pd.read_csv("D:\合作店铺\科帕奇\Agg Data\科帕奇_产品分析.csv",index_col=0)


In [148]:
import mysql.connector

db_host = '192.168.2.108'
db_user= 'jacky'
password= 'jacky_1017'
 
## Database
api_db = 'amazon_swagger'
background_db = 'zonglink_amazon_v9'
currency_db = 'analyze_support_data'

connection = mysql.connector.connect(
            host= db_host,
            database= api_db,
            user= db_user,
            password= password
)


def query_category_data(connection,asin_list):

    ### 从US库里取
    cursor = connection.cursor()
    query = f"""
    SELECT asin,category_root,category_leaf
    FROM zonglink_amazon_v9.tb_product_info_us
    WHERE asin in ('{"', '".join(asin_list)}')"""
    print(query)
    cursor.execute(query)
    query_result = cursor.fetchall()
    query_result = pd.DataFrame(query_result)
    query_result.columns = ['asin','category_root','category_leaf']
    query_result = query_result.drop_duplicates()
    cursor.close()
    
    ### 从EU库里取
    cursor = connection.cursor()
    query_eu = f"""
    SELECT asin,category_root,category_leaf
    FROM zonglink_amazon_v9.tb_product_info_eu
    WHERE asin in ('{"', '".join(asin_list)}')"""
    print(query_eu)
    cursor.execute(query_eu)
    query_result_eu = cursor.fetchall()
    query_result_eu = pd.DataFrame(query_result_eu)
    query_result_eu.columns = ['asin','category_root','category_leaf']
    query_result_eu = query_result_eu.drop_duplicates()

    query_result = pd.concat([query_result,query_result_eu],axis = 0,ignore_index=True)
    query_result = query_result.drop_duplicates()
    return query_result

def Table_5_5_3(product_data): ###主要产品销售分析
    product_data = product_data[(product_data['asin'] != 'Missing')] 

    dates = [datetime.strptime(date, '%Y-%m') for date in product_data.dates]
    max_date = max(dates)
    start_date = max_date - timedelta(days=365)
    selected_dates = [date.strftime('%Y-%m') for date in dates if start_date <= date <= max_date]
    latest_12_df = product_data[product_data['dates'].isin(selected_dates)]
    ### Top 10 产品 
    top_10_asin = latest_12_df.groupby(['asin'])['Total_Sales'].sum().nlargest(10).index  
    top_10 = product_data[(product_data['asin'].isin(top_10_asin))].copy() 
    top_10 = top_10.sort_values(by='asin') 
    asin_df = top_10[['asin']].drop_duplicates()
    ### 上架日期
    earliest_dates = top_10.groupby('asin')['dates'].min() 
    top_10['earliest_date'] = top_10['asin'].map(earliest_dates) 
    earliest_dates_df = top_10[['asin','earliest_date']].drop_duplicates()    
    
    asin_df = pd.merge(asin_df, earliest_dates_df ,on = 'asin',how = 'left') 
    
    ### 2023 年销售额/年销售量/平均单价
    df1 = top_10[(top_10['year'] == 2023)] 
    sales_df = df1.groupby(['seller_id', 'asin']).agg({'Total_Sales': 'sum', 'quantity_shipped': 'sum'}).reset_index() 
    sales_df['Price'] = sales_df['Total_Sales']/sales_df['quantity_shipped']
    asin_df = pd.merge(asin_df, sales_df ,on = 'asin',how = 'left') 

    ### 成本 - 1688

    ### 类别
    asin_list = top_10.asin.unique().tolist()
    print(asin_list)
    category_df = query_category_data(connection, asin_list)
    asin_df = pd.merge(asin_df,category_df,on = 'asin',how = 'left')
    return asin_df

In [149]:
Table_5_5_3(product)

['B01IF4J6TI', 'B07X8WTDXQ', 'B0833WS6D3', 'B09QGPXGB1', 'B0BBTNVRX2', 'B0BG5NT1TQ', 'B0BNWQLM1N', 'B0BP2D9WHF', 'B0BP2F1SQD', 'B0C323QSH9']

    SELECT asin,category_root,category_leaf
    FROM zonglink_amazon_v9.tb_product_info_us
    WHERE asin in ('B01IF4J6TI', 'B07X8WTDXQ', 'B0833WS6D3', 'B09QGPXGB1', 'B0BBTNVRX2', 'B0BG5NT1TQ', 'B0BNWQLM1N', 'B0BP2D9WHF', 'B0BP2F1SQD', 'B0C323QSH9')

    SELECT asin,category_root,category_leaf
    FROM zonglink_amazon_v9.tb_product_info_eu
    WHERE asin in ('B01IF4J6TI', 'B07X8WTDXQ', 'B0833WS6D3', 'B09QGPXGB1', 'B0BBTNVRX2', 'B0BG5NT1TQ', 'B0BNWQLM1N', 'B0BP2D9WHF', 'B0BP2F1SQD', 'B0C323QSH9')


,asin,earliest_date,seller_id,Total_Sales,quantity_shipped,Price,category_root,category_leaf
0,B01IF4J6TI,2022-02,A36JG63EHQR1B6,46766.261931,588.0,79.534459,Industrial & Scientific,Gauges
1,B07X8WTDXQ,2022-02,A36JG63EHQR1B6,67341.728738,194.0,347.122313,Industrial & Scientific,Gauges
2,B07X8WTDXQ,2022-02,A36JG63EHQR1B6,67341.728738,194.0,347.122313,1938713031,5866054031
3,B0833WS6D3,2022-02,A36JG63EHQR1B6,103723.721934,304.0,341.196454,Industrial & Scientific,Gauges
4,B09QGPXGB1,2022-02,A18WUWKDSC6O7I,66240.900000,131.0,505.655725,Automotive,Turbochargers
5,B0BBTNVRX2,2022-10,A2R7HPZ408C2AG,38410.808534,61.0,629.685386,Automotive,Lifters
6,B0BG5NT1TQ,2022-11,A18WUWKDSC6O7I,105977.100000,127.0,834.465354,Automotive,Fuel Injectors
7,B0BNWQLM1N,2023-01,A2R7HPZ408C2AG,54341.589609,105.0,517.538949,Automotive,Code Readers & Scan Tools
8,B0BP2D9WHF,2023-04,AB0LFBRPZ644J,65485.600000,441.0,148.493424,Automotive,Coil Packs
9,B0BP2F1SQD,2023-05,AB0LFBRPZ644J,36171.992071,241.0,150.091253,Automotive,Coil Packs


In [150]:
def Table_5_7_1(product_data):
    product_data['dates'] = pd.to_datetime(product_data['year'].astype(str) + '-' + product_data['month'].astype(str)).dt.strftime('%Y-%m')
    dates = [datetime.strptime(date, '%Y-%m') for date in product_data.dates]
    max_date = max(dates)
    start_date = max_date - timedelta(days=365)
    selected_dates = [date.strftime('%Y-%m') for date in dates if start_date <= date <= max_date]
    latest_12_df = product_data[product_data['dates'].isin(selected_dates)]

    latest_12_df = latest_12_df.groupby(['marketplace','dates'])[['Total_Fee','Total_Tax']].sum().reset_index()
    latest_12_df['Total_Fee'] = latest_12_df['Total_Fee'] + latest_12_df['Total_Tax']
    latest_12_df = latest_12_df.pivot(index='dates',columns='marketplace',values = 'Total_Fee').reset_index()
    latest_12_df['Monthly_Total'] = latest_12_df.iloc[:,1:].sum(axis = 1)
    market_total_row = latest_12_df.iloc[:,1:].sum(axis = 0)
    latest_12_df.loc[len(latest_12_df)] = market_total_row
    return latest_12_df

In [151]:
Table_5_7_1(product)

marketplace,dates,Amazon.ca,Amazon.com,Amazon.com.mx,Monthly_Total
0,2023-02,-4941.503327,-105984.07,-3500.031938,-1.144256e+05
1,2023-03,-7256.339485,-135060.56,-4067.352722,-1.463843e+05
2,2023-04,-8346.876240,-151740.84,-3607.035732,-1.636948e+05
3,2023-05,-9529.441576,-169979.82,-3889.803582,-1.833991e+05
4,2023-06,-10820.977950,-168232.61,-5200.103644,-1.842537e+05
5,2023-07,-10856.830198,-170210.73,-6233.452233,-1.873010e+05
6,2023-08,-12632.569262,-219211.76,-7620.815568,-2.394651e+05
7,2023-09,-15627.126203,-228673.95,-8299.307067,-2.526004e+05
8,2023-10,-15785.771961,-265112.29,-9615.396809,-2.905135e+05
9,2023-11,-14089.331951,-244647.56,-9658.246785,-2.683951e+05


In [130]:
market_total_row = product.iloc[:,1:].sum(axis = 0)

In [152]:
product.groupby(['year'])[['Commission','FBAPerUnitFulfillmentFee','Total_Fee','Total_Tax']].sum()

,Commission,FBAPerUnitFulfillmentFee,Total_Fee,Total_Tax
year,,,,
2022,-4.438482e+05,-144797.290281,-6.067522e+05,-208996.196863
2023,-1.170848e+06,-505367.290464,-1.751855e+06,-629903.319114
2024,-1.263629e+05,-61163.876019,-1.968244e+05,-65447.039998


In [163]:
def Table_5_7_2(product_data):
    table_5_7_2 = product_data.groupby(['year'])[['Commission','FBAPerUnitFulfillmentFee','Total_Fee','Total_Tax']].sum().reset_index()
    table_5_7_2['Other_Fee'] = table_5_7_2['Total_Fee'] - table_5_7_2['Commission'] - table_5_7_2['FBAPerUnitFulfillmentFee']
    table_5_7_2 = table_5_7_2.drop(['Total_Fee'],axis = 1)
    table_5_7_2['Total'] = table_5_7_2.sum(axis = 1)
    table_5_7_2 = table_5_7_2.set_index('year')
    table_5_7_2 = table_5_7_2.transpose()
    return table_5_7_2

In [164]:
Table_5_7_2(product)

year,2022,2023,2024
Commission,-443848.155574,-1.170848e+06,-126362.866445
FBAPerUnitFulfillmentFee,-144797.290281,-5.053673e+05,-61163.876019
Total_Tax,-208996.196863,-6.299033e+05,-65447.039998
Other_Fee,-18106.752536,-7.563967e+04,-9297.701027
Total,-813726.395255,-2.379735e+06,-260247.483489


In [167]:
refund_data = pd.read_csv('D:\合作店铺\科帕奇\Agg Data\科帕奇_退款分析.csv')
table_5_8 = refund_data.groupby(['year_month','market'])[['退单数量','订单总数']].sum().reset_index()

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\jackyzhu\AppData\Local\Temp\ipykernel_45836\3357759341.py:1: SyntaxWarning: invalid escape sequence '\A'
  refund_data = pd.read_csv('D:\合作店铺\科帕奇\Agg Data\科帕奇_退款分析.csv')


In [169]:
table_5_8['退貨率'] = table_5_8['退单数量']/table_5_8['订单总数']

In [178]:
table_5_8.pivot(index='date',values = ['订单总数','退貨率'],columns='market')

订单总数                                退貨率             \
market     Amazon.ca Amazon.com Amazon.com.mx Amazon.ca Amazon.com   
year_month                                                           
2022_10        126.0     2430.0          59.0  0.238095   0.119753   
2022_11        189.0     2792.0          75.0  0.275132   0.114971   
2022_12        143.0     2843.0         110.0  0.293706   0.108688   
2022_2          76.0     1508.0          46.0  0.342105   0.098143   
2022_3         115.0     1867.0          53.0  0.226087   0.098554   
2022_4         138.0     2131.0          39.0  0.326087   0.089160   
2022_5         139.0     2207.0          53.0  0.352518   0.084730   
2022_6         157.0     2201.0          73.0  0.318471   0.089050   
2022_7         209.0     2232.0          91.0  0.191388   0.098118   
2022_8         124.0     2194.0          62.0  0.314516   0.113036   
2022_9         131.0     2358.0          91.0  0.267176   0.104750   
2023_1         195.0     3316.0         105.0  0.323077   0.138721   
2023_10        492.0     8431.0         317.0  0.392276   0.145772   
2023_11        447.0     8004.0         300.0  0.326622   0.140930   
2023_12        405.0     7018.0         216.0  0.303704   0.120690   
2023_2         163.0     3070.0         141.0  0.282209   0.127036   
2023_3         227.0     3942.0         137.0  0.295154   0.135211   
2023_4         263.0     4715.0         128.0  0.296578   0.136585   
2023_5         321.0     5336.0         154.0  0.264798   0.144303   
2023_6         379.0     5152.0         152.0  0.274406   0.146545   
2023_7         372.0     5490.0         203.0  0.298387   0.151366   
2023_8         435.0     6954.0         218.0  0.319540   0.145240   
2023_9         481.0     7421.0         254.0  0.353430   0.152136   
2024_1         418.0     8144.0         230.0  0.107656   0.061272   

                          
market     Amazon.com.mx  
year_month                
2022_10         0.322034  
2022_11         0.266667  
2022_12         0.218182  
2022_2          0.326087  
2022_3          0.188679  
2022_4          0.333333  
2022_5          0.207547  
2022_6          0.191781  
2022_7          0.197802  
2022_8          0.306452  
2022_9          0.318681  
2023_1          0.304762  
2023_10         0.274448  
2023_11         0.466667  
2023_12         0.425926  
2023_2          0.276596  
2023_3          0.145985  
2023_4          0.289062  
2023_5          0.240260  
2023_6          0.440789  
2023_7          0.379310  
2023_8          0.467890  
2023_9          0.444882  
2024_1          0.117391